# Analysis of simulated data


TODO add here a brief description of your project or analysis. Explain the purpose, goals, or context of the notebook here.

## Table of Contents
- [1. Chapter 1 - Introduction](#chapter-1-introduction)
- [2. Chapter 2 - R packages](#chapter-2-R-packages)
- [3. Chapter 3 - Input specification](#chapter-3-Input-specification)
- [4. Chapter 4 - Supermatrix and Supertree trees VS true topologies](#chapter-4-Supermatrix-and-Supertree-trees-VS-true-topologies)
- [5. Chapter 5 - Running times](#chapter-5-Running-times)

<a id="chapter-1-introduction"></a>

## Chapter 1 - Introduction

TODO add description of notebook and how to use it.


<a id="chapter-2-R-packages"></a>

## Chapter 2 - R packages

Here is the list of packages needed for this notebook.

In [41]:
library("phangorn")
library("ggplot2")
library("reshape2")
library("geiger")
library("adephylo")
library("phytools")
library("rlist")
library("plotrix")
library("plyr")
library("dplyr")
library("hrbrthemes")
library("rstatix")
library("gplots")
library("viridis")
library('gtools')
library(gridExtra)
library(grid)
library("ggforce")
library("scales")
library("Biostrings")

TODO make a chapter for aqll custom functions

<a id="chapter-3-Input-specification"></a>

## Chapter 3 - Input specification

Change the parent_of_all_dir variable to give the location of the input. All input files are expected to be under it. Otherwise change the single input location to you liking.<br>
All input files or directory paths are listed here.

In [2]:
# the location of the directory that contains all inputs for analyses 
parent_of_all_dir = "/users/cn/abaltzis/projects/1_paralogs/simulated_datasets/simulated_sequences/"

# The directory (one or more) that contain all the results from the main.nf pipeline run. It can be a glob path.
#dir_containing_all_pipe_results = "../results"
dir_containing_all_pipe_results = paste0(parent_of_all_dir, "rerun_25_species_corrected_branches/rerun_sim_b_factor*")

#  TODO describe what this file does
reference_tree_codefile = file.path(parent_of_all_dir, "ref_tree.code")

# get the species/organism list of names used by the main.nf pipeline
species_name_file = file.path(parent_of_all_dir, "orthologs_org_ids_to_concatenate")


# each run of main.nf has one output directory. here we get that or multiple runs outputs directories.
run_dirs = Sys.glob(dir_containing_all_pipe_results)
run_dirs = run_dirs[file.info(run_dirs)$isdir]

# Create the 'figures' directory if it doesn't exist. Created where this notebook is executed.
if (!dir.exists("figures")) {
    dir.create("figures")
}

<a id="chapter-4-Supermatrix-and-Supertree-trees-VS-true-topologies"></a>

## Chapter 4 - Supermatrix and Supertree trees VS true topologies

TODO add description 

In [174]:
RF_avg_list = list()
RF_se_list = list()
RF_avg_norm_list = list()
collect_branch_len_supermatrix_ml = c()
collect_diameter_supermatrix_ml = c()

# each run of main.nf has one output directory. here we get that or multiple runs outputs directories.
run_dirs = Sys.glob(dir_containing_all_pipe_results)
run_dirs = run_dirs[file.info(run_dirs)$isdir]

for (run in run_dirs) {
    print(basename(run))

    ref_codefile = read.table(file = reference_tree_codefile )
    
    # get al the family subdirectories in each maqin.nf output run dir
    fam_dirs = grep("avg_branchlen*", list.dirs(run, recursive = FALSE), value = TRUE)
    count_fam = 0

    # this will contain the mean branch length for all true/reference paralog trees for each gene family
    avg_branch_length = c()

    # read the names of the ortholog species.
    orgs = read.table(file = species_name_file)

    # get all units trees for all combination of methods 
    for (m in fam_dirs) {
        print(basename(m))
        count_fam = count_fam + 1

        # Get all units trees (250 = 25 organism * 10 samples/statistical repeats) for the combination Minimun Evolution (ME) + supermatrix method (SM). 
        # We unroot every tree and fully resolve it to use the RF.dist package and others. 
        all_units_file  = file.path(m, "results/rerun", "all_units.nwk")
        all_units_trees = read.tree(all_units_file, keep.multi=TRUE)
        all_units_trees = unroot(all_units_trees)
        
        # check if there any non-bynary (consensus trees) among the 250 trees present in the file above
        if (length(which(is.binary(all_units_trees) == FALSE)) != 0) {
            
            # if there are non-binary trees, iterate through them and transformed into a binary one using multi2di package
            for (tree in which(is.binary(all_units_trees) == FALSE)) {
                all_units_trees[[tree]] = multi2di(all_units_trees[[tree]])
            }
        }

        # Get all units trees (250) ME + supertree (ST) method and handle non-binary trees as above
        all_superfine_file  = file.path(m, "results/rerun", "all_units_superfine_trees.nwk")
        all_superfine_trees = read.tree(all_superfine_file, keep.multi=TRUE)
        all_superfine_trees = unroot(all_superfine_trees)
        if (length(which(is.binary(all_superfine_trees) == FALSE)) != 0) {
            for (tree in which(is.binary(all_superfine_trees) == FALSE)) {
                all_superfine_trees[[tree]] = multi2di(all_superfine_trees[[tree]])
            }
        }

        # Get all units trees (250) Maximum Likelihood (ML) + SM 
        all_units_raxml_file  = file.path(m, "results/rerun", "all_units_RAxML_trees.nwk")
        all_units_raxml_trees = read.tree(all_units_raxml_file, keep.multi=TRUE)
        all_units_raxml_trees = unroot(all_units_raxml_trees)
        if (length(which(is.binary(all_units_raxml_trees) == FALSE)) != 0) {
            for (tree in which(is.binary(all_units_raxml_trees) == FALSE)) {
                all_units_raxml_trees[[tree]] = multi2di(all_units_raxml_trees[[tree]])
            }
        }

        # take only the last 10 of the 250 above trees (unit25's 10 samples/replicates)(unit25 = all 25 single units concatenated) and collect branch lengths (each branch in tree, 27 in total 2*15-3) and diameter (using patristic distance)
        for(mytree in 241:length(all_units_raxml_trees)) {
            collect_branch_len_supermatrix_ml = c(collect_branch_len_supermatrix_ml, all_units_raxml_trees[[mytree]]$edge.length)
            collect_diameter_supermatrix_ml = c(collect_diameter_supermatrix_ml, max(distTips(all_units_raxml_trees[[mytree]], method = 'patristic')))
        }

        # Get all units trees (250) ML + ST
        all_raxml_superfine_file  = file.path(m, "results/rerun", "all_units_RAxML_superfine_trees.nwk")
        all_raxml_superfine_trees = read.tree(all_raxml_superfine_file, keep.multi=TRUE)
        all_raxml_superfine_trees = unroot(all_raxml_superfine_trees)
        if (length(which(is.binary(all_raxml_superfine_trees) == FALSE)) != 0) {
            for (tree in which(is.binary(all_raxml_superfine_trees) == FALSE)) {
                all_raxml_superfine_trees[[tree]] = multi2di(all_raxml_superfine_trees[[tree]])
            }
        }

        # Reference tree block, each gene family has it's own reference tree that is the one generated by the simulation step. Along this tree all sequences are generated and then alligned and passed as input for main.nf pipeline.
        ref_tree_file = file.path(run, paste0(basename(m), ".nwk"))
        ref_tree      = read.tree(ref_tree_file, keep.multi=FALSE)

        # Since each gene family has a predifined gene tree (ref_tree) composed of a fixed species tree pasted as leaf of the paralog tree. The tree is designed like this by choic4. Each gene families varies only in the paralog tree that it has that is unique (15 tips).
        # Here we extract the paralog tree and to do so we just need to extract the subtree with all the sequences of omne species. Since each species have the same paralog tree. So we can extract the subtree of whichever species it does not matter.
        ref_subtree   = keep.tip(ref_tree,as.character(ref_codefile$V1))

        # we then rename the tips of the paralog tree (ref_subtree) to a more generic keyname. Effectively going from rat1 to seq1. 
        for (j in 1:length(ref_subtree$tip.label)) { 
            ref_subtree$tip.label[j] = as.character(ref_codefile$V2[which(ref_codefile$V1 == ref_subtree[["tip.label"]][j])])
        }

        # Unroot the tree and perform aother operation and then save the paralog tree to the specifc gene faimily directory
        ref_subtree        = unroot(ref_subtree)
        ref_subtree        = list(ref_subtree)
        class(ref_subtree) = "multiPhylo"
        subtree_out_file   = file.path(m, paste0("ref_subtree_", basename(m), ".nwk"))
        write.tree(ref_subtree, file=subtree_out_file)

        # update on the average branch length with the mean of the true/refernce paralog tree of the given gene family
        avg_branch_length = c(avg_branch_length, mean(ref_subtree[[1]]$edge.length))

        # Make a list containing all trees gathered so far and compute all possible pair comparisons among trees. There are 1001*1001-1001 unique pairewise comparisons ( a square matrix of size 1001 with 0 in diagonal ).
        # On this matrix (plotdata) all other measuramnts are done. This is a symmetric matrix (distance matrix)
        #all_trees        = c(ref_subtree, all_units_trees, all_superfine_trees, all_units_raxml_trees, all_raxml_superfine_trees)
        all_trees        = c(ref_subtree, all_units_trees[seq(1, 250, 10)], all_superfine_trees[seq(1, 250, 10)], all_units_raxml_trees[seq(1, 250, 10)], all_raxml_superfine_trees[seq(1, 250, 10)])
        class(all_trees) = "multiPhylo"
        all_vs_all_rf    = RF.dist(all_trees,normalize=TRUE)
        plotdata         = data.matrix(all_vs_all_rf)

        # all ME metrics to be computed
        mean_rf           = c()
        se_rf             = c()
        mean_rf_norm      = c()
        mean_superfine_rf = c()
        se_superfine_rf   = c()

        # all ML metrics to be computed
        mean_raxml_rf           = c()
        se_raxml_rf             = c()
        mean_raxml_rf_norm      = c()
        mean_raxml_superfine_rf = c()
        se_raxml_superfine_rf   = c()

        # proceed throught the plotdata matrix taking from the second element one every ten up to 242. Basically proceed at chunkcs of 10 (samples). On those chuncks compute the mean and the Standard error. 
        # average all sample "values" whithin a unit. going from 250 value to 25 averages
        # This iterate only through the values of the comparison between the reference paralofg tree (ref_subtree)(first column/row of matrix) and the 250 paraogs trees of ME + SM (all_units_trees)
        for (i in seq(2, (length(orgs$V1)*10)+1, by=10)) {
            mean_rf = c(mean_rf, mean(plotdata[1, i:(i+9)]))
            se_rf   = c(se_rf, (sd(plotdata[1, i:(i+9)])/sqrt(10)))
        }

        # This instead goes from row 252 to 492 of the plotdata matrix. It still goes in chuncks of 10 (samples). 
        # So this effectively does what is done above but between the reference paralofg tree and the 250 paraogs trees of ME + ST (all_superfine_trees)
        for (i in seq( (length(orgs$V1)*10)+2, (2*(length(orgs$V1)*10)+1), by=10 )) {
            mean_superfine_rf = c(mean_superfine_rf, mean(plotdata[1,i:(i+9)]))
            se_superfine_rf   = c(se_superfine_rf, (sd(plotdata[1,i:(i+9)])/sqrt(10)))
        }

        # This for loop goes from 502 to 742. It still goes in chuncks of 10 (samples).
        # So this effectively does what is done above but between the reference paralofg tree and the 250 paraogs trees of ML + SM (all_units_raxml_trees)
        for (i in seq( (2*(length(orgs$V1)*10))+2, (3*(length(orgs$V1)*10)+1) ,by=10 )) {
            mean_raxml_rf = c(mean_raxml_rf, mean(plotdata[1,i:(i+9)]))
            se_raxml_rf   = c(se_raxml_rf, (sd(plotdata[1,i:(i+9)])/sqrt(10)))
        }

        # This for loop goes from 752 to 992. It still goes in chuncks of 10 (samples).
        # So this effectively does what is done above but between the reference paralofg tree and the 250 paraogs trees of ML + ST (all_raxml_superfine_trees)
        for (i in seq( (3*(length(orgs$V1)*10))+2, (4*(length(orgs$V1)*10)+1), by=10)) {
            mean_raxml_superfine_rf = c(mean_raxml_superfine_rf, mean(plotdata[1,i:(i+9)]))
            se_raxml_superfine_rf   = c(se_raxml_superfine_rf, (sd(plotdata[1,i:(i+9)])/sqrt(10)))
        }

        # TODO finish this analysis
        
    }
}

[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species"
[1] ".nextflow"


Warning message in file(file, "r"):
“cannot open file '/users/cn/abaltzis/projects/1_paralogs/simulated_datasets/simulated_sequences/rerun_25_species_corrected_branches/rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species/.nextflow/results/rerun/all_units.nwk': No such file or directory”


ERROR: Error in file(file, "r"): cannot open the connection


In [72]:
plotdata

,1,2,3,4,5,6,7,8,9,10,⋯,32,33,34,35,36,37,38,39,40,41
1,0.0000000,0.3333333,0.41666667,0.33333333,0.25000000,0.25000000,0.33333333,0.33333333,0.25000000,0.33333333,⋯,0.5000000,0.25000000,0.25000000,0.25000000,0.33333333,0.25000000,0.33333333,0.16666667,0.33333333,0.16666667
2,0.3333333,0.0000000,0.41666667,0.33333333,0.25000000,0.25000000,0.33333333,0.33333333,0.25000000,0.33333333,⋯,0.3333333,0.25000000,0.33333333,0.33333333,0.41666667,0.25000000,0.41666667,0.16666667,0.41666667,0.16666667
3,0.4166667,0.4166667,0.00000000,0.08333333,0.16666667,0.16666667,0.08333333,0.08333333,0.16666667,0.08333333,⋯,0.4166667,0.16666667,0.33333333,0.25000000,0.33333333,0.16666667,0.41666667,0.25000000,0.41666667,0.25000000
4,0.3333333,0.3333333,0.08333333,0.00000000,0.08333333,0.08333333,0.00000000,0.00000000,0.08333333,0.00000000,⋯,0.4166667,0.08333333,0.25000000,0.16666667,0.25000000,0.08333333,0.33333333,0.16666667,0.33333333,0.16666667
5,0.2500000,0.2500000,0.16666667,0.08333333,0.00000000,0.00000000,0.08333333,0.08333333,0.00000000,0.08333333,⋯,0.4166667,0.00000000,0.16666667,0.08333333,0.16666667,0.00000000,0.25000000,0.08333333,0.25000000,0.08333333
6,0.2500000,0.2500000,0.16666667,0.08333333,0.00000000,0.00000000,0.08333333,0.08333333,0.00000000,0.08333333,⋯,0.4166667,0.00000000,0.16666667,0.08333333,0.16666667,0.00000000,0.25000000,0.08333333,0.25000000,0.08333333
7,0.3333333,0.3333333,0.08333333,0.00000000,0.08333333,0.08333333,0.00000000,0.00000000,0.08333333,0.00000000,⋯,0.4166667,0.08333333,0.25000000,0.16666667,0.25000000,0.08333333,0.33333333,0.16666667,0.33333333,0.16666667
8,0.3333333,0.3333333,0.08333333,0.00000000,0.08333333,0.08333333,0.00000000,0.00000000,0.08333333,0.00000000,⋯,0.4166667,0.08333333,0.25000000,0.16666667,0.25000000,0.08333333,0.33333333,0.16666667,0.33333333,0.16666667
9,0.2500000,0.2500000,0.16666667,0.08333333,0.00000000,0.00000000,0.08333333,0.08333333,0.00000000,0.08333333,⋯,0.4166667,0.00000000,0.16666667,0.08333333,0.16666667,0.00000000,0.25000000,0.08333333,0.25000000,0.08333333
10,0.3333333,0.3333333,0.08333333,0.00000000,0.08333333,0.08333333,0.00000000,0.00000000,0.08333333,0.00000000,⋯,0.4166667,0.08333333,0.25000000,0.16666667,0.25000000,0.08333333,0.33333333,0.16666667,0.33333333,0.16666667


<a id="chapter-5-Running-times"></a>

## Chapter 5 - Running times

TODO add description 

In [42]:
# read the trace files for running times
for (run in run_dirs) {
    print(basename(run))

    # each main.nf output run has a trace file repoprting various information on the run itself
    tracefile_path = file.path(run, "trace_cpu_time.txt")
    tracefile      = read.table(tracefile_path, header = TRUE)
    
    # here we parse the trace file to retain only the processes that we want to take the running time (realtime) from
    tracefile_mod  = tracefile[grep("simulated_data:run_phylo_ML_supermatrix_aln_sim|simulated_data:only_concatenate_aln_sim|simulated_data:run_phylo_ML_full_aln_sim", tracefile$native_id), c("name", "native_id", "realtime")]
    
    # transforming the above table to one that have the process names (simulated_data:run_phylo_ML_supermatrix_aln_sim ecc..) as column names, 
    # the rows will be still the family names (working as ID) and the values in each column will be the running times associated to the column name (process name) for that given family.
    tracefile_mod_wide = spread(tracefile_mod, native_id, realtime)

    # creating a new last column called Supermatrix_ML that has aas values the running times of concatenation plus ML tree computation (on concatenated MSA) 
    tracefile_mod_wide$Supermatrix_ML = as.numeric(tracefile_mod_wide$`simulated_data:only_concatenate_aln_sim`) + as.numeric(tracefile_mod_wide$`simulated_data:run_phylo_ML_supermatrix_aln_sim`)

    # retain only the family name, BIgTree ML running time and SM + ML running time
    tracefile_mod_wide = tracefile_mod_wide[,c(1,3,5)]

    # call the second column name Big_Tree and set their value to numeric type
    colnames(tracefile_mod_wide)[2] = "Big_Tree"
    tracefile_mod_wide$Big_Tree = as.numeric(tracefile_mod_wide$Big_Tree)

    # adding a first column that groups families by the run of the main.nf
    tracefile_mod_wide$family_group = rep(basename(run), 100)

    # add tables toghether raw-wise
    if(run == run_dirs[1]) {
        fin_tracefile_mod_wide = tracefile_mod_wide
    } else {
        fin_tracefile_mod_wide = rbind(fin_tracefile_mod_wide, tracefile_mod_wide)
    }
}

[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species"
[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_17th_20th_22th_and_23th_50_fams_25_species"
[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_20th_22th_and_23th_50_fams_25_species"
[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_22th_and_23th_50_fams_25_species"
[1] "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_23th_50_fams_25_species"


In [168]:
# TODO make nicer and update with whatever is changed above

# make the means of all (500) family running time for the BigTree and ML + SM. Bring them to seconds (/1000) and round the digits to 2
round(colMeans(fin_tracefile_mod_wide[, c(2, 3)])/1000,digits = 2)

# compute the median for the BigTree approach and the  ML + SM.
round(median(fin_tracefile_mod_wide$Big_Tree)/1000,digits = 2)
round(median(fin_tracefile_mod_wide$Supermatrix_ML)/1000,digits = 2)

Big_Tree Supermatrix_ML 
        638.12          67.76

[1] 588.3

[1] 63.85

### Figure 2

TODO add chapter link and description if necessary.

In [169]:
#
# Cpu time vs Method (SM + ML and Bigtree)
#

# It takes a table of all families as rows (500) and it goes to a table of 1000 (500 BigTree + 500 SM + ML) where the running times instead of being separated into two different coluns now they are in the same one.
# name + group columns will work as unique ID.  group is the directory name of the main.nf run/s, while name is the name of the family. 
fin_tracefile_mod_wide_melted           = melt(fin_tracefile_mod_wide)

# transform running times from millisecond to seconds. And rename the column names.
fin_tracefile_mod_wide_melted$value     = fin_tracefile_mod_wide_melted$value / 1000
colnames(fin_tracefile_mod_wide_melted) = c("name", "family_group", "Method", "Time")

# crate a table with two boxplots (x axis) with Method columns as values and on the y the running times.
# the plot is not showed because this file was executed on a cluster and rendering of plots creates conflicts with the kernel. 
p = ggplot(fin_tracefile_mod_wide_melted,aes(x=Method, y=Time, fill=Method)) +
    geom_boxplot(show.legend = FALSE) +
    theme_light() +
    ylab("CPU time (sec)") + xlab("Method")
ggsave(filename="figures/CPU_time_per_method.png", plot=p, dpi = "retina")

Using name, family_group as id variables

Saving 7 x 7 in image


### todo make the follewing plot block either disappear or better.

In [4]:
# TODO make this into the section of custom functions.
# Function to assign new values based on a mapping provided by the user
assign_mapped_values <- function(df, column_name, mapping, new_col_name) {
  # Check if mapping is a named vector or a list
  if (!is.vector(mapping) || is.null(names(mapping))) {
    stop("Mapping must be a named vector or list.")
  }
  
  # Get the unique values from the specified column
  unique_values = unique(df[[column_name]])
  
  # Check if the mapping contains all unique values
  if (!all(unique_values %in% names(mapping))) {
    stop("Mapping must include all unique values from the specified column.")
  }
  
  # Initialize a vector to store the assigned values
  assigned_values = vector("character", nrow(df))
  
  # Assign values based on the mapping
  for (i in seq_along(unique_values)) {
    assigned_values[df[[column_name]] == unique_values[i]] <- mapping[names(mapping) == unique_values[i]]
  }
  
  # Add the assigned values to the original dataframe
  df[[new_col_name]] = assigned_values
  
  return(df)
}

In [5]:
#
# Cpu time vs Method (SM + ML and Bigtree) divided per group (by sequence length)
#

# assign to each family_group a mapping values  (as new column). basically insert the length of the sequences in each family as an additional column in the dataframe.
# Remember each family in the same group have the same length of sequence generated, the MSA though might differ in length due to gaps.
mapping = c("rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_23th_50_fams_25_species" = "group1\nlen = 32",
            "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_22th_and_23th_50_fams_25_species" = "group2\nlen = 46",
            "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_20th_22th_and_23th_50_fams_25_species" = "group3\nlen = 50",
            "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_17th_20th_22th_and_23th_50_fams_25_species" = "group4\nlen = 60",
            "rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species" = "group5\nlen = 82")
fin_tracefile_mod_reassigned = assign_mapped_values(fin_tracefile_mod_wide, "family_group", mapping, "length")

# reorder the columns to have the numerical values at the end
fin_tracefile_mod_reassigned = fin_tracefile_mod_reassigned[, c("name", "family_group", "length", "Big_Tree", "Supermatrix_ML")]

# It takes a table of all families as rows (500) and it goes to a table of 1000 (500 BigTree + 500 SM + ML) where the running times instead of being separated into two different coluns now they are in the same one.
# name + group columns will work as unique ID.  group is the directory name of the main.nf run/s, while name is the name of the family. 
fin_tracefile_mod_melted           = melt(fin_tracefile_mod_reassigned)


# transform running times from millisecond to seconds. And rename the column names.
fin_tracefile_mod_melted$value     = fin_tracefile_mod_melted$value / 1000
colnames(fin_tracefile_mod_melted) = c("name", "family_group", "length", "Method", "Time")

# create a table with 5 pairs of boxplot. Group1, 2.. ecc.. each group has 100 families and two running time (realtime) mesuraments for the BigTree and SM + ML approaches.
# the plot is not showed because this file was executed on a cluster and rendering of plots creates conflicts with the kernel.
p = ggplot(fin_tracefile_mod_melted, aes(x=length, y=Time, fill=Method)) +
    geom_boxplot(show.legend = TRUE) +
    theme_light() +
    labs(x = "sequence length",
       y = "CPU time (sec)",
       fill = "Method")
ggsave(filename="figures/CPU_time_per_group_BigTree_SM_ML.png", plot=p, dpi = "retina")

Using name, family_group, length as id variables

Saving 7 x 7 in image


### TODO make the following better or remove it

In [48]:
# update the trace matrix with the length of the input msa

fin_tracefile_mod_with_len = fin_tracefile_mod_wide

# for each family go and read the input MSA (input of main.nf) to get length of MSA (number of columns).
# Iterate over each row of the dataframe
for (i in 1:nrow(fin_tracefile_mod_with_len)) {
    # Construct the full file path using group (directory) and name (filename) to read the input msa
    # TODO make it not depend on the rerun_bla_bla direcrtory
    input_msa_path = file.path(paste0(parent_of_all_dir, "rerun_25_species_corrected_branches"), fin_tracefile_mod_with_len$family_group[i], paste0(gsub("[()]", "", fin_tracefile_mod_with_len$name[i]), ".ma"))

    # read the msa in fasta format and get the length of the first sequence (aka length of the alignment)
    msa_file = readAAStringSet(input_msa_path)
    msa_len  = length(msa_file[[1]])
    
    # add the length to the table 
    fin_tracefile_mod_with_len$length[i] = msa_len
}

In [49]:
#
# Cpu time vs Method (SM + ML and Bigtree) log ratio plot of each family (x axis number columns BigMSA) 
# 

# Create a new column with the division of Big_Tree by Supermatrix_ML running times for each family (500)
# and then remove the original columns Big_Tree Supermatrix_ML
fin_tracefile_mod_ratio = fin_tracefile_mod_with_len %>%
  mutate(fold_ratio = Big_Tree / Supermatrix_ML) %>%
  select(-Big_Tree, -Supermatrix_ML)

# create a table with on x axis the length of the MSA (n columns)  and on y the fold ratio (BigTree / SM + ML) of running times. The plot will have now 500 points.
# the plot is not showed because this file was executed on a cluster and rendering of plots creates conflicts with the kernel.
p = ggplot(fin_tracefile_mod_ratio, aes(x=length, y=fold_ratio)) +
    geom_point(color = 'blue', size = 2, alpha = 0.6) +  # Add points
    theme_light() +
    labs(x = "MSA length (n# columns)",
       y = "fold ratio (BigTree / Supermatrix_ML)")
ggsave(filename="figures/CPU_time_scatterplot_msa_len_BigTree_SM_ML.png", plot=p, dpi = "retina")

name,family_group,length,fold_ratio
<chr>,<chr>,<int>,<dbl>
(avg_branchlen_0.7_protein_family_001.sub),rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species,87,9.424765
(avg_branchlen_0.7_protein_family_002.sub),rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species,90,6.662016
(avg_branchlen_0.7_protein_family_003.sub),rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species,89,13.213900
(avg_branchlen_0.7_protein_family_004.sub),rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species,95,8.258667
(avg_branchlen_0.7_protein_family_005.sub),rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species,87,12.687788
(avg_branchlen_0.7_protein_family_006.sub),rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species,87,8.113131
(avg_branchlen_0.7_protein_family_007.sub),rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species,86,8.950375
(avg_branchlen_0.7_protein_family_008.sub),rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species,87,9.403475
(avg_branchlen_0.7_protein_family_009.sub),rerun_sim_b_factor_3.5_avg_branchlen_0.7_from_11th_17th_20th_22th_and_23th_50_fams_25_species,87,7.888725


Saving 7 x 7 in image


### TODO the below was just a 10 familoy trial make it go away 

In [109]:
# TODO make this into the section of custom functions.
# Function to convert to milliseconds
convert_to_ms <- function(time_str) {
    # Extract components using regular expressions
    millis = "0"
    seconds = "0"
    minutes = "0"
    if (str_detect(time_str, "\\d+(\\.\\d+)?(?=ms)")) {
        millis <- str_extract(time_str, "\\d+(\\.\\d+)?(?=ms)")
    } else {
        seconds <- str_extract(time_str, "\\d+(\\.\\d+)?(?=s)")
        minutes <- str_extract(time_str, "\\d+(\\.\\d+)?(?=m)")
    }
    
    # Convert each component to milliseconds (coercing to numeric where needed)
    seconds_ms <- as.numeric(seconds) * 1000
    minutes_ms <- as.numeric(minutes) * 60 * 1000
    millis_ms <- as.numeric(millis)
    
    # Sum all components and handle NAs by treating them as 0
    total_ms <- sum(c(seconds_ms, minutes_ms, millis_ms), na.rm = TRUE)
    return(total_ms)
}

In [120]:
for (run in run_dirs) {
    print(basename(run))

    # each main.nf output run has a trace file repoprting various information on the run itself
    tracefile_path = file.path(run, "pipeline_info/execution_trace_2024-09-27_17-39-02.txt")
    tracefile      = read.table(tracefile_path, sep = "\t", header = TRUE)

    # here we parse the trace file to retain only the processes that we want to take the running time (realtime) from
    tracefile_mod = tracefile[grep("simulated_data:run_phylo_ML_supermatrix_aln_sim|simulated_data:only_concatenate_aln_sim|simulated_data:run_phylo_ML_full_aln_sim", tracefile$name), c("name", "realtime")]

    # Use strsplit to split the 'name' column by whitespace
    split_name    = strsplit(as.character(tracefile_mod$name), " ")

    # Create new columns
    tracefile_mod$name       = sapply(split_name, `[`, 2) # First part 
    tracefile_mod$native_id  = sapply(split_name, `[`, 1) # Second part 

    # Reorder columns so 'native_id' becomes the second column
    tracefile_mod = tracefile_mod[, c("name", "native_id", "realtime")]

    # trasform from human readable time to millisecond time
    tracefile_mod$realtime = sapply(tracefile_mod$realtime, convert_to_ms)

    # transforming the above table to one that have the process names (simulated_data:run_phylo_ML_supermatrix_aln_sim ecc..) as column names, 
    # the rows will be still the family names (working as ID) and the values in each column will be the running times associated to the column name (process name) for that given family.
    tracefile_mod_wide = spread(tracefile_mod, native_id, realtime)
    
    # creating a new last column called Supermatrix_ML that has aas values the running times of concatenation plus ML tree computation (on concatenated MSA) 
    tracefile_mod_wide$Supermatrix_ML = as.numeric(tracefile_mod_wide$`simulated_data:only_concatenate_aln_sim`) + as.numeric(tracefile_mod_wide$`simulated_data:run_phylo_ML_supermatrix_aln_sim`)

    # retain only the family name, BIgTree ML running time and SM + ML running time
    tracefile_mod_wide = tracefile_mod_wide[,c(1,3,5)]

    # call the second column name Big_Tree and set their value to numeric type
    colnames(tracefile_mod_wide)[2] = "Big_Tree"
    tracefile_mod_wide$Big_Tree = as.numeric(tracefile_mod_wide$Big_Tree)

    # adding a first column that groups families by the run of the main.nf
    tracefile_mod_wide$group = rep(basename(run), 10)

    # add tables toghether raw-wise
    if(run == run_dirs[1]) {
        fin_tracefile_mod_wide = tracefile_mod_wide
    } else {
        fin_tracefile_mod_wide = rbind(fin_tracefile_mod_wide, tracefile_mod_wide)
    }
}

[1] "results"


In [121]:
# make the means of all (500) family running time for the BigTree and ML + SM. Bring them to seconds (/1000) and round the digits to 2
round(colMeans(fin_tracefile_mod_wide[, c(2, 3)])/1000,digits = 2)

# compute the median for the BigTree approach and the  ML + SM.
round(median(fin_tracefile_mod_wide$Big_Tree)/1000,digits = 2)
round(median(fin_tracefile_mod_wide$Supermatrix_ML)/1000,digits = 2)

Big_Tree Supermatrix_ML 
        195.40          23.47

[1] 196

[1] 21.87

In [122]:

fin_tracefile_mod_wide_melted           = melt(fin_tracefile_mod_wide)

fin_tracefile_mod_wide_melted$value     = fin_tracefile_mod_wide_melted$value / 1000
colnames(fin_tracefile_mod_wide_melted) = c("name","group","Method","Time")

p = ggplot(fin_tracefile_mod_wide_melted,aes(x=Method,y=Time,fill=Method)) +
    geom_boxplot(show.legend = FALSE) +
    theme_light() +
    ylab("CPU time (sec)") + xlab("Method")
ggsave(filename="CPU_time_per_method_10_fam.png", plot=p, dpi = "retina")

Using name, group as id variables

Saving 7 x 7 in image
